In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure
from skimage.io import imread, imshow
from skimage import img_as_ubyte
from skimage.color import rgb2gray
from skimage.exposure import histogram, cumulative_distribution
from scipy.stats import cauchy, logistic

In [ ]:
def linear_distribution(image, channel):
    image_intensity = img_as_ubyte(image[:,:,channel])
    freq, _ = cumulative_distribution(image_intensity)
    target_bins = np.arange(255)
    target_freq = np.linspace(0, 1, len(target_bins))
    new_vals = np.interp(freq, target_freq, target_bins)
    return new_vals[image_intensity].astype(np.uint8)

In [ ]:
# image = cv2.imread("data/2135041001/front.jpg")
# image = cv2.imread("data/2135041006/front.jpg")
# image = cv2.imread("data/2135041007/front.jpg")
# image = cv2.imread("data/2135041040/front.jpg") 
# image = cv2.imread("data/2135041043/front.jpg") # edges got blended into cases
# image = cv2.imread("data/2135041048/front.jpg") # edges got blended into cases
# image = cv2.imread("data/2135041054/front.jpg")
# image = cv2.imread("data/3718372019/front.jpg")
# image = cv2.imread("data/3734194001/front.jpg")
# image = cv2.imread("data/3761772053/front.jpg") # edges got blended into cases
# image = cv2.imread("data/3788404014/front.jpg")
# image = cv2.imread("data/3798764016/front.jpg") # not found
# image = cv2.imread("data/3821556106/front.jpg") # edges got blended into cases
# image = cv2.imread("data/3824670033/front.jpg") # edges got blended into cases
# image = cv2.imread("data/3838512098/front.jpg") # not found
# image = cv2.imread("data/3865838027/front.jpg") # not found
# image = cv2.imread("data/3871141001/front.jpg") # edges got blended into cases
image = cv2.imread("data/3836849030/front.jpg") # edges got blended into cases
#workflow

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# to grayscale image
gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

# get edges
blur = cv2.GaussianBlur(gray, (3, 3), -10)
adaptive_binary = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 7,3)
edges = cv2.Canny(adaptive_binary,100,200)
binarized_grad = 255 - edges

# denoises again
open_binarized_grad = cv2.morphologyEx(
    binarized_grad, 
    cv2.MORPH_OPEN, 
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7)))

# get contours
contours, _ = cv2.findContours(open_binarized_grad, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

# image size
height, width = binarized_grad.shape
image_area = height * width

# sort contour index
index_sort = sorted(range(len(contours)), key=lambda i : cv2.contourArea(contours[i]),reverse=True)
contours_sort = [contours[i] for i in index_sort]

# get area and perimeter
contour_area = [cv2.contourArea(contours_sort[i]) for i in range(len(index_sort))]
contour_peri = [cv2.arcLength(contours_sort[i], True) for i in range(len(index_sort))]
approx = [cv2.approxPolyDP(contours_sort[i], 0.001 * contour_peri[i], True) for i in range(len(index_sort))]
bounding_box = [cv2.boundingRect(approx[i]) for i in range(len(index_sort))]
is_card = list(filter(lambda x : x >= 0, [i if contour_area[i] >= 0.48 * image_area and contour_area[i] <= 0.6 * image_area else -1 for i in range(len(index_sort))]))

is_card_index = is_card[-1]
card_bounding_box = bounding_box[is_card_index]

final = cv2.rectangle(
    image.copy(), 
    (int(card_bounding_box[0]), int(card_bounding_box[1])),
    (int(card_bounding_box[0] + card_bounding_box[2]), int(card_bounding_box[1] + card_bounding_box[3])), 
    (0,255,0), 3
)

In [ ]:
plt.imshow(image)

In [ ]:
# convert to gray and increase intensity
plt.imshow(gray, cmap = 'gray')

In [ ]:
plt.imshow(binarized_grad, cmap = 'gray')

In [ ]:
plt.imshow(open_binarized_grad, cmap = 'gray')

In [ ]:
plt.imshow(final)

In [ ]:
def test_contour(image : np.ndarray):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # to grayscale image
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # get edges
    blur = cv2.GaussianBlur(gray, (3, 3), -10)
    adaptive_binary = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 7,3)
    edges = cv2.Canny(adaptive_binary,100,200)
    binarized_grad = 255 - edges

    # denoises again
    open_binarized_grad = cv2.morphologyEx(
        binarized_grad, 
        cv2.MORPH_OPEN, 
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7)))

    # get contours
    contours, _ = cv2.findContours(open_binarized_grad, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    # image size
    height, width = binarized_grad.shape
    image_area = height * width

    # sort contour index
    index_sort = sorted(range(len(contours)), key=lambda i : cv2.contourArea(contours[i]),reverse=True)
    contours_sort = [contours[i] for i in index_sort]

    # get area and perimeter
    contour_area = [cv2.contourArea(contours_sort[i]) for i in range(len(index_sort))]
    contour_peri = [cv2.arcLength(contours_sort[i], True) for i in range(len(index_sort))]
    approx = [cv2.approxPolyDP(contours_sort[i], 0.001 * contour_peri[i], True) for i in range(len(index_sort))]
    bounding_box = [cv2.boundingRect(approx[i]) for i in range(len(index_sort))]
    is_card = list(filter(lambda x : x >= 0, [i if contour_area[i] >= 0.48 * image_area and contour_area[i] <= 0.6 * image_area else -1 for i in range(len(index_sort))]))

    return len(is_card) > 0

In [3]:
import glob
import os
total_number_of_images = 0
contour_found = 0
for file_path in glob.glob(os.path.join('data','*')):
    image_file_path = os.path.join(file_path, "front.jpg")
    try:
        image = cv2.imread(image_file_path)
        if test_contour(image):
            contour_found += 1
        total_number_of_images += 1
    except:
        pass
    print(f"Current images whose content found: {contour_found}")
    print(f"Current processed images: {total_number_of_images}")

Current images whose content found: 4437
Current processed images: 5343
Current images whose content found: 4437
Current processed images: 5344
Current images whose content found: 4438
Current processed images: 5345
Current images whose content found: 4439
Current processed images: 5346
Current images whose content found: 4440
Current processed images: 5347
Current images whose content found: 4441
Current processed images: 5348
Current images whose content found: 4441
Current processed images: 5349
Current images whose content found: 4442
Current processed images: 5350
Current images whose content found: 4443
Current processed images: 5351
Current images whose content found: 4444
Current processed images: 5352
Current images whose content found: 4445
Current processed images: 5353
Current images whose content found: 4445
Current processed images: 5354
Current images whose content found: 4446
Current processed images: 5355
Current images whose content found: 4447
Current processed image